Montar servidor HTTP usando flask y MySQL

In [ ]:
sudo apt-get update

In [ ]:
sudo apt-get install mysql-server

In [ ]:
sudo systemctl start mysql

In [ ]:
sudo systemctl enable mysql

Acceder a MySQL como usuario root

In [ ]:
sudo mysql -u root -p

Crear Base de Datos, usarla y crear Tabla

In [ ]:
CREATE DATABASE emqx_data CHARACTER SET utf8mb4;

In [ ]:
USE emqx_data;

In [ ]:
CREATE TABLE luces_iot (
  id INT AUTO_INCREMENT PRIMARY KEY,
  dispositivo_id VARCHAR(255),
  estado VARCHAR(255),
  marca_tiempo TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

Crear usuario para usar bd

In [ ]:
CREATE USER 'alumno'@'localhost' IDENTIFIED BY 'querty';

In [ ]:
GRANT ALL PRIVILEGES ON emqx_data.* TO 'alumno'@'localhost';

In [ ]:
FLUSH PRIVILEGES;

Instalar python y flask

In [ ]:
sudo apt install python3

In [ ]:
pip install flask

In [ ]:
pip install mysql-connector-python

Crear carpeta y archivo para servidor

In [ ]:
cd http_server

In [ ]:
touch http_server.py

In [ ]:
nano http_server.py

In [ ]:
En el editor nano se necesita poner el código que configura el servidor, es el siguiente:

In [ ]:
import mysql.connector
from flask import Flask, json, request

api = Flask(__name__)

# Establece la conexión a tu base de datos MySQL
db_connection = mysql.connector.connect(
    host="localhost",
    user="alumno",
    password="querty",
    database="emqx_data"
)

@api.route('/', methods=['POST'])
def print_messages():
    reply = {"result": "ok", "message": "success"}
    data = request.get_data().decode('utf-8')
    print("got post request: ", data)

    # Convierte la cadena JSON a un objeto Python
    json_data = json.loads(data)

    # Inserta los datos en la tabla "luces_iot" en la base de datos "emqx_data"
    cursor = db_connection.cursor()
    query = "INSERT INTO luces_iot (dispositivo_id, estado) VALUES (%s, %s)"
    values = (json_data["dispositivo_id"], json_data["estado"])
    cursor.execute(query, values)
    db_connection.commit()
    cursor.close()

    return json.dumps(reply), 200

@api.route('/consultar', methods=['GET'])
def consultar_bd():
    cursor = db_connection.cursor(dictionary=True)
    query = "SELECT * FROM luces_iot"
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()

    return json.dumps(result), 200

if __name__ == '__main__':
    api.run(host='0.0.0.0', port=5000)


Salir con ctrl + x y guardar cambios
Luego ejecutar el siguiende comando para iniciar el servidor

In [ ]:
python3 http_server.py

Probar con postman y Hacer data bridge

Ejemplo de json para el data bridge

In [ ]:
{
    "dispositivo_id": "led1",
    "estado": "rojo"
}

Ejemplo de la sentencia para la regla

In [ ]:
SELECT * FROM "led" WHERE payload = 'rojo'